In [1]:
from email.policy import default
from typing import List  # isort:skip
import re
from typing import Dict, List
from collections import Counter

from requesting_urls import get_html
from collections import defaultdict
import requests
import concurrent.futures

def better_articles(html):
  base_url = "https://en.wikipedia.org"
  articles = set()
  long_pat = re.compile(r"(?<=href=\")((http\:\/\/en\.wikipedia[^#\\\"]+)|(\/wiki[^#\"]+))")
  
  for article in long_pat.findall(html):
    if article is not None:
      if (article[0][0] == "/"):
        article = base_url + article[0]
      else:
        article = article[0]
      if not ":" in article[6:]:
        articles.add(article)
  
  return articles

class Node:
  def __init__(self, data):
    self.data = data
    self.last = None

class Graph:

  def __init__(self, target):
    self.graph = defaultdict(list)
    self.trail = {}
    self.target = target
    self.count = 0
    self.visited = defaultdict(lambda: False)

  def extendGraph(self, node):
    html: str = get_html(node)
    articles = better_articles(html)
    for article in articles:
      if article != node:
        self.addEdge(node, article)

  def addEdge(self, u, v):
    self.graph[u].append(v)
    if not v in self.trail:
      node = Node(v)
      self.trail[v] = node
      node.last = self.trail[u]

  def move_through_queue(self, url):
    self.queue.remove(url)
    self.extendGraph(url)
    self.count += 1
    print(self.count, url)
    path = []
    find = False
    for i in self.graph[url]:
      if i == self.target:
        print("Found it here!")
        path = [i]
        breadcrumbs = self.trail[i]
        while breadcrumbs.last != None:
          path.insert(0, breadcrumbs.last.data)
          breadcrumbs = breadcrumbs.last
        print(path)
        find = True
      if self.visited[i] == False:
        self.queue.append(i)
        self.visited[i] = True
    return url, path, find

  def Search(self, start):
    self.trail[start] = Node(start)
    
    self.queue = []
    count = 0

    self.queue.append(start)
    self.visited[start] = True

    while True:
      with concurrent.futures.ProcessPoolExecutor(max_workers=1) as executor:
        futures = [executor.submit(self.move_through_queue, url) for url in self.queue]
        for future in concurrent.futures.as_completed(futures):
          url, path, find = future.result()
          print(count, url)
          if find:
            return path
        #     node = queue.pop(0)
        #     self.extendGraph(node)
        #     count += 1
        #     print(count, node)
        #     for i in self.graph[node]:
              
        #       if i == self.target:
        #         print("Found it here!")
        #         path = [i]
        #         breadcrumbs = self.trail[i]
        #         while breadcrumbs.last != None:
        #           path.insert(0, breadcrumbs.last.data)
        #           breadcrumbs = breadcrumbs.last

        #         print(path)
        #         return path
        #       if visited[i] == False:
        #         queue.append(i)
        #         visited[i] = True
      
          
def find_path(start: str, finish: str) -> List[str]:
    """Find the shortest path from `start` to `finish`

    Arguments:
      start (str): wikipedia article URL to start from
      finish (str): wikipedia article URL to stop at

    Returns:
      urls (list[str]):
        List of URLs representing the path from `start` to `finish`.
        The first item should be `start`.
        The last item should be `finish`.
        All items of the list should be URLs for wikipedia articles.
        Each article should have a direct link to the next article in the list.
    """
    graph = Graph(finish)
    path = graph.Search(start)
    assert path[0] == start
    assert path[-1] == finish
    return path

In [2]:
start = "https://en.wikipedia.org/wiki/The_Emoji_Movie"
finish = "https://en.wikipedia.org/wiki/Vietnam_War"

In [3]:
%load_ext line_profiler

In [6]:
import asyncio
import logging
import re
import sys
from typing import IO
import urllib.error
import urllib.parse

import aiofiles
import aiohttp
from aiohttp import ClientSession

In [11]:
logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(name)s: %(message)s",
    level=logging.DEBUG,
    datefmt="%H:%M:%S",
    stream=sys.stderr,
)
logger = logging.getLogger("areq")
logging.getLogger("chardet.charsetprober").disabled = True
async def fetch_html(url: str, session: ClientSession, **kwargs) -> str:
    """GET request wrapper to fetch page HTML.

    kwargs are passed to `session.request()`.
    """

    resp = await session.request(method="GET", url=url, **kwargs)
    resp.raise_for_status()
    logger.info("Got response [%s] for URL: %s", resp.status, url)
    html = await resp.text()
    return html

async def parse(url: str, session: ClientSession, **kwargs) -> set:
    """Find HREFs in the HTML of `url`."""
    found = set()
    try:
        html = await fetch_html(url=url, session=session, **kwargs)
    except (
        aiohttp.ClientError,
        aiohttp.http_exceptions.HttpProcessingError,
    ) as e:
        logger.error(
            "aiohttp exception for %s [%s]: %s",
            url,
            getattr(e, "status", None),
            getattr(e, "message", None),
        )
        return found
    except Exception as e:
        logger.exception(
            "Non-aiohttp exception occured:  %s", getattr(e, "__dict__", {})
        )
        return found
    else:
        base_url = "https://en.wikipedia.org"
        long_pat = re.compile(r"(?<=href=\")((http\:\/\/en\.wikipedia[^#\\\"]+)|(\/wiki[^#\"]+))")
        
        for article in long_pat.findall(html):
            if article is not None:
                if (article[0][0] == "/"):
                    article = base_url + article[0]
                else:
                    article = article[0]
                if not ":" in article[6:]:
                    found.add(article)
        return found

async def bulk_crawl_and_write(file: IO, urls: set, **kwargs) -> None:
    """Crawl & write concurrently to `file` for multiple `urls`."""
    async with ClientSession() as session:
        tasks = []
        for url in urls:
            tasks.append(
                write_one(file=file, url=url, session=session, **kwargs)
            )
        await asyncio.gather(*tasks)



In [18]:
find_path(start, finish)

AttributeError: Can't pickle local object 'Graph.__init__.<locals>.<lambda>'

In [1]:
import asyncio
import random
import time


async def worker(name, queue):
    while True:
        # Get a "work item" out of the queue.
        sleep_for = await queue.get()

        # Sleep for the "sleep_for" seconds.
        await asyncio.sleep(sleep_for)

        # Notify the queue that the "work item" has been processed.
        queue.task_done()

        print(f'{name} has slept for {sleep_for:.2f} seconds')


async def main():
    # Create a queue that we will use to store our "workload".
    queue = asyncio.Queue()

    # Generate random timings and put them into the queue.
    total_sleep_time = 0
    for _ in range(20):
        sleep_for = random.uniform(0.05, 1.0)
        total_sleep_time += sleep_for
        queue.put_nowait(sleep_for)

    # Create three worker tasks to process the queue concurrently.
    tasks = []
    for i in range(3):
        task = asyncio.create_task(worker(f'worker-{i}', queue))
        tasks.append(task)

    # Wait until the queue is fully processed.
    started_at = time.monotonic()
    await queue.join()
    total_slept_for = time.monotonic() - started_at

    # Cancel our worker tasks.
    for task in tasks:
        task.cancel()
    # Wait until all worker tasks are cancelled.
    await asyncio.gather(*tasks, return_exceptions=True)

    print('====')
    print(f'3 workers slept in parallel for {total_slept_for:.2f} seconds')
    print(f'total expected sleep time: {total_sleep_time:.2f} seconds')


asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop